Lab 4 Activity

In [5]:
import pandas as pd
import numpy as np
from IPython.display import Image
import random
from random import randrange

np.set_printoptions(precision = 3)

In [2]:
data_pd = pd.read_csv("jester-data-1.csv", index_col=False, usecols=range(1,101))
d = data_pd.to_latex()
text_file = open("Jester.txt", "w")
text_file.write(d)
text_file.close()

Data pre-processing

In [16]:
data = data_pd.values
train_set = data.copy()
test_samples = []

for i in range(int(data.shape[0]*0.1)):
    x = randrange(data.shape[0])
    y = randrange(data.shape[1])
    while data[x,y] == 99 and (x, y) in test_samples:
        x = randrange(data.shape[0])
        y = randrange(data.shape[1])
    test_samples.append((x,y))
    train_set[x,y] = 99
    

In [17]:
n_features = 2

user_ratings = data_pd.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

Recommender Algorithm

In [18]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(ratings, iterations = 300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = ratings[user_id][item_id]
                if(rating != 99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%1000 == 0 ):
            print(mse)

Training

In [25]:
sgd(train_set)

22.831858147152566


KeyboardInterrupt: 

Predictions

In [20]:
predictions = latent_user_preferences.dot(latent_item_features.T)

Measures

In [26]:
error = []

for (x, y) in test_samples:
    diff = data[x,y]-predictions[x,y]
    error.append(diff)
    
mse = (np.array(error)**2).mean()

print("MSE", mse)


MSE 2589.6251103676054
